In [0]:
# gold_aggregate_vc.py
from pyspark.sql.functions import count, sum as sum_, col

# Load the silver table as a DataFrame
silver_investments_vc = spark.table("workspace_elena_vc.default.silver_investments_vc")

"""
create Gold DataFrame with agg metrics:
    - amount of invests for each investor
    - count of investment for each investor
"""
gold_investments_vc = (
        silver_investments_vc.groupBy("investor_name")
                 .agg(
                     count("investment_id").alias("total_investments"),
                     sum_("investment_amount").alias("total_amount")
                 )
                 .orderBy(col("total_amount").desc())
    )

gold_investments_vc.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace_elena_vc.default.gold_investments_vc")

